In [3]:
from engine.torchengine import AnalysisFunction, Function, EliminateAnalysisMergeResiduals, ElimResidual
import torch
torch.manual_seed(1);

In [4]:
a1 = (('x2', 'x3'), ('x1','x4'), lambda a, b: (sum(a)+b, a[:2]+b))
a2 = (('x1', 'x3'), ('x5',), lambda a, b: (a@b,))
r = (('x4', 'x5'), lambda a, b: torch.cat((a, b)))
indices = {
    'x1': [0,1],
    'x2': [2,3,4],
    'x3': [5,6],
    'x4': [7,8],
    'x5': [9]
}
# update indices to have tensor entries instead of lists
for k,v in indices.items():
    indices[k] = torch.tensor(v)

In [5]:
A1 = AnalysisFunction(a1, indices)
A2 = AnalysisFunction(a2, indices)
R = Function(r, indices)
S = EliminateAnalysisMergeResiduals([A1, A2], [R])
ER = ElimResidual(S, solvefor=['x2'], indices=indices)

In [6]:
x0 = torch.rand(10, dtype=torch.float64) #Use 64 bit floats for better precision and gradients to work
# Fix parameters (x3)
x0[indices['x3']] = torch.tensor([1., 2], dtype=torch.float64)
# Set initial values for solve vars (x2)
x0[indices['x2']] = torch.tensor([1, 1, 1], dtype=torch.float64)

In [7]:
ER(x0)

tensor([ 0.0611,  0.2246, -1.0000, -2.0000,  1.3333,  1.0000,  2.0000,  0.7084,
         0.5798,  0.4967], dtype=torch.float64)

In [8]:
# Clone with requires grad for gradient calculations
xvar = x0.clone()
xvar.requires_grad_()
xout = ER(xvar)
backpass = torch.zeros_like(xvar)
backpass[indices['x2'][2]] = 1
xout.backward(backpass)

In [9]:
xvar.grad.numpy()

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.88888889, 0.22222222, 0.        , 0.        , 0.        ])

## Sellar

In [12]:
from engine.torchengine import AnalyticalSetSympy, FunctionSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, ElimResidual
from engine.torchdata import generate_optim_functions, symbols
import torch
import sympy as sp
torch.manual_seed(1);
import numpy as np
np.set_printoptions(formatter={'float': lambda x: 
                                f"{x:.3f}".rstrip('0').rstrip('.')})

In [13]:
x1,x2,x3,u1,u2, indices = symbols('x1,x2,x3,u1,u2', dim='scalar')

In [14]:
a1 = AnalyticalSetSympy(x1**2 + x2 + x3 - 0.2*u2, u1, indices)
a2 = AnalyticalSetSympy(u1**0.5 + x1 + x2, u2, indices)
objf = FunctionSympy(x3**2 + x2 + u1 + sp.exp(-u2), indices)
g1 = FunctionSympy(3.16-u1, indices)
g2 = FunctionSympy(u2-24, indices)
g = EliminateAnalysisMergeResiduals(functions=[g1,g2])

### MDF

In [15]:
S1 = EliminateAnalysisMergeResiduals(functions=[a1.residual, a2.residual]) # Newton / Jacobi
S2 = EliminateAnalysisMergeResiduals([a1.analysis], functions=[a2.residual]) # GS order 1
S3 = EliminateAnalysisMergeResiduals([a2.analysis], functions=[a1.residual]) # GS order 2
E1 = ElimResidual(S1, solvefor=[a1.outputvar, a2.outputvar], indices=indices)
E2 = ElimResidual(S2, solvefor=[a2.outputvar], indices=indices)
E3 = ElimResidual(S3, solvefor=[a1.outputvar], indices=indices)

In [16]:
x0 = torch.tensor([0,0,1,0.1,0.1])
E3(x0)
x0

tensor([0.0000, 0.0000, 1.0000, 0.8190, 0.1000])

In [17]:
F1 = EliminateAnalysis([E1], [objf, g])
F2 = EliminateAnalysis([E2, a1.analysis], [objf, g])
F3 = EliminateAnalysis([E3, a2.analysis], [objf, g])

In [18]:
x0 = torch.tensor([5.,2.,1.,1.,1.], dtype=torch.float64)
solver_indices = F1.structure[0]
xguess, obj, ineq, eq, dobj, dineq, deq = generate_optim_functions(
    F1, solver_indices, x0, inequality_direction='positive-null', 
    residuals=False, equalities=False)
constraints = [{'type': 'ineq', 'fun': ineq, 'jac': dineq}]

In [19]:
from scipy import optimize
optimize.minimize(obj, xguess, jac=dobj, constraints=constraints,
                  bounds=[(0,10), (0,10), (0,10)])

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.1833939517268077
       x: [ 1.978e+00  0.000e+00  4.227e-15]
     nit: 6
     jac: [ 3.508e+00  1.729e+00  9.405e-01]
    nfev: 6
    njev: 6

In [20]:
x0.detach().numpy()

array([1.978, 0, 0, 3.16, 3.755])